This file tests the one layer NN vs a direct linear classifier using GloVe vectors to predict the 8 hatebase features. We define a new cost function that is similar to AUC but deals with the 8 features separately.

In [1]:
from tf_custom_models import OneLayerNNRetrofit, SoftmaxClassifier
from utility import train_and_eval_auc, HATEBASE_FIELDS
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import roc_auc_score as AUC

import matplotlib.pyplot as plt

import os
from os.path import join as pjoin

from tqdm import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
import logging
import json
import itertools
import scipy
from operator import itemgetter

In [2]:
DATA_DIR = "data/hatebase"
EMBEDDING_SIZE = 100
STATE_SIZE = 100
GLOVE_SIZE = 1193514
GLOVE_PATH = "data/glove/glove.twitter.27B.%dd.txt" % EMBEDDING_SIZE

EMBED_PATH = "data/hatebase/embeddings.%dd.dat" % EMBEDDING_SIZE
HIDDEN_EMBED_PATH = "data/hatebase/embeddings.hidden.%dd.dat" % EMBEDDING_SIZE
NEW_EMBED_PATH = "data/hatebase/embeddings.new.%dd.dat" % EMBEDDING_SIZE
HB_PATH = "data/hatebase/lexicon.csv"
VOCAB_PATH = "data/hatebase/vocab.dat"

In [3]:
def load_embeddings(embed_path, vocab, force=False):
    if not os.path.exists(embed_path):
        hb_vecs = np.zeros((len(vocab), EMBEDDING_SIZE))
        with open(GLOVE_PATH, 'r') as fh:
            found = []
            for line in tqdm(fh, total=GLOVE_SIZE):
                array = line.strip().split(" ")
                word = array[0]
                if word in vocab:
                    idx = vocab[word]
                    found.append(idx)
                    vector = list(map(np.float64, array[1:]))
                    hb_vecs[idx, :] = vector
            # words not found are set to random values
            unfound = list(set(vocab.values()) - set(found))
            for i in unfound:
                hb_vecs[i, :] = np.random.randn(EMBEDDING_SIZE)
                
        hb_vecs = pd.DataFrame(hb_vecs)
        hb_vecs.to_csv(embed_path, header = False, index = False)
        return hb_vecs

    with open(embed_path, 'rb') as embed_path:
        data_x = pd.read_csv( embed_path, header = None, quoting = 0, dtype = np.float64 )
        return data_x

In [4]:
def get_multioutput_y(data_y):
    # y transformed from [[0,0,1],[1,0,1]] => [[[1,0],[0,1]],...]
    # one hots in pos i of each vec form y_i
    y = []
    for i in range(len(HATEBASE_FIELDS)):
        y.append([np.eye(2)[vec[i]] for vec in data_y.values])
    return y

In [5]:
# grab the data
hatebase_data = pd.read_csv( HB_PATH, header = 0, index_col = 0, quoting = 0, 
                                dtype = HATEBASE_FIELDS, usecols = range(len(HATEBASE_FIELDS)+1) )
vocab = dict([(x, y) for (y, x) in enumerate(hatebase_data.index)])
hatebase_embeddings = load_embeddings(EMBED_PATH, vocab, True)

train_i, test_i = train_test_split( np.arange( len( hatebase_embeddings )), train_size = 0.8, random_state = 44 )
#train_x = hatebase_embeddings.ix[train_i].values

# need to preserve order because vocab
train_x = hatebase_embeddings.values
test_x = hatebase_embeddings.ix[test_i].values
train_y = get_multioutput_y(hatebase_data)
test_y = get_multioutput_y(hatebase_data.ix[test_i])

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [6]:
def train_and_eval_auc( train_x, train_y, test_x, test_y, model ):
    model.fit( train_x, train_y )
    p = model.predict_proba( test_x )
    p = p[:,1] if p.shape[1] > 1 else p[:,0]

    auc = AUC( test_y, p )
    print "AUC:", auc

In [7]:
tf.reset_default_graph()
nn = OneLayerNNRetrofit(h=200, max_iter=4000, retrofit_iter=4000)
new_embeddings = nn.fit( train_x, train_y )

[[ 0.77337989 -0.06057805  0.79156985 ...,  0.52147999  0.16876012
  -0.34863999]
 [ 0.65194843  0.19998563  0.38786194 ..., -0.06480067 -0.30253828
  -0.45807016]
 [ 0.5094349   0.54910205 -1.6684682  ..., -0.00378645  0.44879726
   0.31851168]
 ..., 
 [-0.03507394 -0.70313003 -0.10693002 ...,  0.66183986  0.3275601
   0.35258008]
 [ 0.2923187   1.26268643  0.91365821 ...,  0.18256398  0.41281006
  -0.86940039]
 [ 0.4053499   0.15776577  0.26044003 ...,  0.52545592 -0.47759021
   0.64162532]]
[[ 0.77337977 -0.0605781   0.7915697  ...,  0.52147998  0.16876023
  -0.34863999]
 [ 0.65194849  0.1999856   0.38786189 ..., -0.06480076 -0.30253816
  -0.45807009]
 [ 0.50943496  0.54910201 -1.66846821 ..., -0.00378656  0.44879735
   0.31851176]
 ..., 
 [-0.03507388 -0.70313005 -0.10693004 ...,  0.66183973  0.3275602
   0.35258016]
 [ 0.29231876  1.2626864   0.91365819 ...,  0.18256387  0.41281017
  -0.86940032]
 [ 0.40534997  0.15776575  0.26043999 ...,  0.52545583 -0.47759012
   0.64162541]]
[[

[[ 0.77337757 -0.0605788   0.79156711 ...,  0.52148033  0.16876203
  -0.34864007]
 [ 0.65194948  0.19998519  0.38786112 ..., -0.06480207 -0.30253638
  -0.45806902]
 [ 0.50943585  0.54910139 -1.66846826 ..., -0.003788    0.44879871
   0.31851315]
 ..., 
 [-0.03507301 -0.70313038 -0.10693028 ...,  0.66183784  0.32756184
   0.35258155]
 [ 0.29231961  1.26268604  0.91365788 ...,  0.1825623   0.41281195
  -0.86939909]
 [ 0.40535109  0.1577655   0.26043949 ...,  0.52545456 -0.47758866
   0.64162698]]
[[ 0.77337744 -0.06057883  0.79156697 ...,  0.52148037  0.16876212
  -0.34864008]
 [ 0.65194953  0.19998518  0.38786108 ..., -0.06480213 -0.30253629
  -0.45806896]
 [ 0.50943589  0.54910137 -1.66846826 ..., -0.00378805  0.44879877
   0.31851322]
 ..., 
 [-0.03507297 -0.7031304  -0.10693029 ...,  0.66183776  0.32756192
   0.35258162]
 [ 0.29231964  1.26268602  0.91365787 ...,  0.18256223  0.41281204
  -0.86939903]
 [ 0.40535114  0.15776549  0.26043947 ...,  0.52545451 -0.47758859
   0.64162706]]


[[ 0.77337526 -0.06057928  0.79156484 ...,  0.52148128  0.16876343
  -0.3486404 ]
 [ 0.65195024  0.19998497  0.38786059 ..., -0.06480285 -0.30253501
  -0.45806811]
 [ 0.50943647  0.54910104 -1.66846824 ..., -0.00378878  0.44879971
   0.31851422]
 ..., 
 [-0.0350724  -0.70313054 -0.10693036 ...,  0.66183671  0.32756314
   0.35258273]
 [ 0.29232019  1.26268589  0.91365769 ...,  0.18256138  0.41281338
  -0.86939804]
 [ 0.40535187  0.15776544  0.26043924 ...,  0.5254539  -0.47758746
   0.6416283 ]]
[[ 0.77337512 -0.0605793   0.79156471 ...,  0.52148135  0.1687635
  -0.34864042]
 [ 0.65195028  0.19998496  0.38786056 ..., -0.06480289 -0.30253493
  -0.45806805]
 [ 0.5094365   0.54910103 -1.66846823 ..., -0.00378882  0.44879977
   0.31851428]
 ..., 
 [-0.03507237 -0.70313054 -0.10693037 ...,  0.66183666  0.32756321
   0.35258279]
 [ 0.29232022  1.26268588  0.91365769 ...,  0.18256134  0.41281346
  -0.86939798]
 [ 0.40535191  0.15776544  0.26043923 ...,  0.52545387 -0.47758739
   0.64162837]]
[

[[ 0.77337226 -0.06057971  0.79156222 ...,  0.52148291  0.16876493
  -0.34864101]
 [ 0.65195103  0.19998483  0.38786013 ..., -0.06480347 -0.30253354
  -0.45806706]
 [ 0.50943709  0.54910078 -1.66846813 ..., -0.00378938  0.44880074
   0.31851537]
 ..., 
 [-0.03507177 -0.7031306  -0.10693034 ...,  0.66183578  0.32756455
   0.35258406]
 [ 0.29232077  1.26268586  0.91365756 ...,  0.18256067  0.41281494
  -0.86939683]
 [ 0.40535262  0.15776553  0.2604391  ...,  0.52545347 -0.47758607
   0.64162982]]
[[ 0.77337212 -0.06057973  0.7915621  ...,  0.52148299  0.16876499
  -0.34864104]
 [ 0.65195107  0.19998483  0.38786011 ..., -0.06480349 -0.30253348
  -0.45806701]
 [ 0.50943712  0.54910077 -1.66846812 ..., -0.0037894   0.44880079
   0.31851542]
 ..., 
 [-0.03507174 -0.7031306  -0.10693034 ...,  0.66183574  0.32756461
   0.35258412]
 [ 0.29232079  1.26268586  0.91365755 ...,  0.18256064  0.41281501
  -0.86939677]
 [ 0.40535266  0.15776554  0.2604391  ...,  0.52545346 -0.47758601
   0.64162989]]


[[ 0.77336934 -0.06058002  0.79155983 ...,  0.52148471  0.16876622
  -0.34864172]
 [ 0.65195173  0.19998479  0.3878598  ..., -0.06480386 -0.3025323
  -0.45806611]
 [ 0.50943763  0.54910064 -1.66846797 ..., -0.00378974  0.4488016
   0.31851634]
 ..., 
 [-0.03507122 -0.70313057 -0.10693023 ...,  0.66183517  0.32756578
   0.35258526]
 [ 0.29232125  1.26268595  0.91365748 ...,  0.18256023  0.4128163
  -0.86939575]
 [ 0.40535324  0.15776574  0.26043906 ...,  0.52545329 -0.4775848
   0.64163121]]
[[ 0.77336919 -0.06058004  0.79155972 ...,  0.5214848   0.16876629
  -0.34864176]
 [ 0.65195176  0.19998479  0.38785979 ..., -0.06480387 -0.30253224
  -0.45806606]
 [ 0.50943765  0.54910063 -1.66846796 ..., -0.00378976  0.44880164
   0.31851639]
 ..., 
 [-0.0350712  -0.70313057 -0.10693023 ...,  0.66183514  0.32756584
   0.35258532]
 [ 0.29232127  1.26268595  0.91365748 ...,  0.18256021  0.41281636
  -0.86939569]
 [ 0.40535327  0.15776576  0.26043906 ...,  0.52545328 -0.47758474
   0.64163128]]
[[ 0

...

[[ 0.77275106 -0.06054628  0.79121343 ...,  0.52194172  0.16895834
  -0.34899294]
 [ 0.65211231  0.20003271  0.3878342  ..., -0.06481016 -0.30241014
  -0.45789806]
 [ 0.50953276  0.54913366 -1.66842451 ..., -0.00379173  0.4488714
   0.31864492]
 ..., 
 [-0.03496439 -0.70307811 -0.10690884 ...,  0.66182372  0.32769603
   0.3527756 ]
 [ 0.29239623  1.26275243  0.91364767 ...,  0.18255738  0.41294383
  -0.86924233]
 [ 0.40543007  0.15786958  0.26041907 ...,  0.52547845 -0.4774286
   0.64184574]]
[[ 0.77275089 -0.06054627  0.79121335 ...,  0.52194184  0.16895839
  -0.34899306]
 [ 0.65211236  0.20003272  0.38783418 ..., -0.06481017 -0.30241012
  -0.457898  ]
 [ 0.50953278  0.54913367 -1.66842451 ..., -0.00379173  0.44887141
   0.31864495]
 ..., 
 [-0.03496436 -0.7030781  -0.10690884 ...,  0.66182371  0.32769605
   0.35277565]
 [ 0.29239625  1.26275245  0.91364766 ...,  0.18255737  0.41294385
  -0.86924229]
 [ 0.40543009  0.15786961  0.26041905 ...,  0.52547845 -0.47742857
   0.64184579]]
[[

[[ 0.77274764 -0.06054598  0.79121187 ...,  0.52194413  0.16895944
  -0.34899545]
 [ 0.65211334  0.20003294  0.38783391 ..., -0.0648103  -0.3024097
  -0.45789702]
 [ 0.5095333   0.54913381 -1.66842438 ..., -0.00379179  0.44887161
   0.31864563]
 ..., 
 [-0.03496379 -0.70307787 -0.10690894 ...,  0.66182359  0.3276965
   0.35277663]
 [ 0.29239662  1.26275269  0.91364752 ...,  0.18255729  0.41294424
  -0.86924156]
 [ 0.40543044  0.15787004  0.26041872 ...,  0.52547845 -0.47742804
   0.64184679]]
[[ 0.77274747 -0.06054597  0.79121179 ...,  0.52194425  0.16895949
  -0.34899558]
 [ 0.65211339  0.20003295  0.38783389 ..., -0.06481031 -0.30240968
  -0.45789697]
 [ 0.50953332  0.54913382 -1.66842437 ..., -0.00379179  0.44887162
   0.31864566]
 ..., 
 [-0.03496376 -0.70307786 -0.10690894 ...,  0.66182358  0.32769653
   0.35277668]
 [ 0.29239664  1.2627527   0.91364752 ...,  0.18255729  0.41294426
  -0.86924153]
 [ 0.40543046  0.15787006  0.2604187  ...,  0.52547845 -0.47742801
   0.64184684]]
[[

[[ 0.77274422 -0.06054568  0.7912103  ...,  0.52194653  0.16896053
  -0.34899798]
 [ 0.65211438  0.20003318  0.38783362 ..., -0.06481045 -0.30240927
  -0.45789598]
 [ 0.50953384  0.54913395 -1.66842425 ..., -0.00379184  0.44887181
   0.31864633]
 ..., 
 [-0.03496319 -0.70307763 -0.10690905 ...,  0.66182346  0.32769697
   0.35277766]
 [ 0.292397    1.26275294  0.91364737 ...,  0.1825572   0.41294464
  -0.8692408 ]
 [ 0.40543081  0.15787049  0.26041836 ...,  0.52547844 -0.47742749
   0.64184784]]
[[ 0.77274405 -0.06054566  0.79121023 ...,  0.52194665  0.16896059
  -0.34899811]
 [ 0.65211443  0.20003319  0.38783361 ..., -0.06481046 -0.30240925
  -0.45789593]
 [ 0.50953386  0.54913396 -1.66842424 ..., -0.00379184  0.44887182
   0.31864637]
 ..., 
 [-0.03496316 -0.70307762 -0.10690905 ...,  0.66182345  0.327697
   0.35277771]
 [ 0.29239702  1.26275295  0.91364737 ...,  0.1825572   0.41294466
  -0.86924076]
 [ 0.40543083  0.15787051  0.26041835 ...,  0.52547844 -0.47742746
   0.64184789]]
[[

Iteration 4000: loss: 1.88059885808 

[[ 0.7727408  -0.06054537  0.79120874 ...,  0.52194894  0.16896163
  -0.34900052]
 [ 0.65211542  0.20003341  0.38783333 ..., -0.0648106  -0.30240884
  -0.45789494]
 [ 0.50953438  0.5491341  -1.66842412 ..., -0.0037919   0.44887201
   0.31864704]
 ..., 
 [-0.03496259 -0.7030774  -0.10690915 ...,  0.66182333  0.32769744
   0.35277869]
 [ 0.29239739  1.26275319  0.91364723 ...,  0.18255711  0.41294505
  -0.86924004]
 [ 0.40543119  0.15787094  0.26041801 ...,  0.52547844 -0.47742693
   0.64184889]]
[[ 0.77274063 -0.06054536  0.79120867 ...,  0.52194906  0.16896169
  -0.34900064]
 [ 0.65211547  0.20003342  0.38783332 ..., -0.06481061 -0.30240882
  -0.45789488]
 [ 0.50953441  0.54913411 -1.66842411 ..., -0.0037919   0.44887202
   0.31864708]
 ..., 
 [-0.03496256 -0.70307739 -0.10690916 ...,  0.66182332  0.32769747
   0.35277874]
 [ 0.29239741  1.2627532   0.91364722 ...,  0.18255711  0.41294507 -0.86924   ]
 [ 0.4054312   0.15787096  0.26041799 ...,  0.52547844 -0.47742691
   0.64184894]]
[[

In [8]:
def cosine(u, v):        
    """Cosine distance between 1d np.arrays `u` and `v`, which must have 
    the same dimensionality. Returns a float."""
    # Use scipy's method:
    return scipy.spatial.distance.cosine(u, v)
    # Or define it yourself:
    # return 1.0 - (np.dot(u, v) / (vector_length(u) * vector_length(v)))

def neighbors(word, mat, rownames, distfunc=cosine):    
    """Tool for finding the nearest neighbors of `word` in `mat` according 
    to `distfunc`. The comparisons are between row vectors.
    
    Parameters
    ----------
    word : str
        The anchor word. Assumed to be in `rownames`.
        
    mat : np.array
        The vector-space model.
        
    rownames : list of str
        The rownames of mat.
            
    distfunc : function mapping vector pairs to floats (default: `cosine`)
        The measure of distance between vectors. Can also be `euclidean`, 
        `matching`, `jaccard`, as well as any other distance measure  
        between 1d vectors.
        
    Raises
    ------
    ValueError
        If word is not in rownames.
    
    Returns
    -------    
    list of tuples
        The list is ordered by closeness to `word`. Each member is a pair 
        (word, distance) where word is a str and distance is a float.
    
    """
    if word not in rownames:
        raise ValueError('%s is not in this VSM' % word)
    w = mat[rownames.index(word)]
    dists = [(rownames[i], distfunc(w, mat[i])) for i in range(len(mat))]
    return sorted(dists, key=itemgetter(1), reverse=False)

In [21]:
print hatebase_embeddings.values

[[ 0.77336499 -0.06043783  0.79164391 ...,  0.52155562  0.1684498
  -0.34873804]
 [ 0.65202285  0.19990168  0.38775393 ..., -0.06487487 -0.30257175
  -0.45801132]
 [ 0.50945705  0.54905885 -1.66846575 ..., -0.00383365  0.44877673
   0.31854086]
 ..., 
 [-0.0350304  -0.70317717 -0.10698134 ...,  0.66176233  0.32752223
   0.35261874]
 [ 0.2923489   1.26261222  0.91360889 ...,  0.18250047  0.41278971
  -0.86934098]
 [ 0.40540092  0.15771362  0.26040549 ...,  0.52539517 -0.47761454
   0.6416618 ]]


In [8]:
 0.77337989 -0.06057805  0.79156985 ...,  0.52147999  0.16876012
  -0.34863999]

NameError: name 'new_embeddings' is not defined

In [20]:
print neighbors("bitch", hatebase_embeddings.values, list(hatebase_data.index), cosine)[:5]
print neighbors("hoe", hatebase_embeddings.values, list(hatebase_data.index), cosine)[:5]
print neighbors("bitch", new_embeddings, list(hatebase_data.index), cosine)[:5]
print neighbors("hoe", new_embeddings, list(hatebase_data.index), cosine)[:5]

[('bitch', -2.2204460492503131e-16), ('nigga', 0.12581809454602721), ('hoe', 0.157471676977334), ('niggah', 0.24974724411391869), ('pussy', 0.25110263162451019)]
[('hoe', 0.0), ('bitch', 0.157471676977334), ('nigga', 0.23986537405048902), ('nigger', 0.27917007496991209), ('niggah', 0.28542226847984054)]
[('bitch', -2.2204460492503131e-16), ('nigga', 0.12581809454602721), ('hoe', 0.157471676977334), ('niggah', 0.24974724411391869), ('pussy', 0.25110263162451019)]
[('hoe', 0.0), ('bitch', 0.157471676977334), ('nigga', 0.23986537405048902), ('nigger', 0.27917007496991209), ('niggah', 0.28542226847984054)]


In [12]:
probs = nn.predict_proba( test_x )

total_auc = 0
for i in range(len(probs)):
    # hack to get the positive class
    y = [s[1] for s in test_y[i]]
    y_pred = [s[1] for s in probs[i]]
    auc = AUC( y, y_pred )
    print HATEBASE_FIELDS.keys()[i], "AUC:", auc
    total_auc += auc
print "Average AUC:", total_auc/len(probs)

about_religion AUC: 0.692660550459
about_gender AUC: 0.832467532468
about_nationality AUC: 0.714285714286
about_class AUC: 0.552115987461
about_ethnicity AUC: 0.630662020906
about_sexual_orientation AUC: 0.666666666667
about_disability AUC: 0.724137931034
Average AUC: 0.687570914754


In [13]:
# write hidden states
new_embeddings = pd.DataFrame(new_embeddings)
new_embeddings.to_csv(NEW_EMBED_PATH, header = False, index = False)

1000 retrofit iters

``
about_religion AUC: 0.418577981651
about_gender AUC: 0.67012987013
about_nationality AUC: 0.691071428571
about_class AUC: 0.496865203762
about_ethnicity AUC: 0.498954703833
about_sexual_orientation AUC: 0.631578947368
about_disability AUC: 0.448275862069
Average AUC: 0.550779142484
``

2000 retrofit iters

``
about_religion AUC: 0.43004587156
about_gender AUC: 0.555844155844
about_nationality AUC: 0.8125
about_class AUC: 0.524686520376
about_ethnicity AUC: 0.571428571429
about_sexual_orientation AUC: 0.570175438596
about_disability AUC: 0.534482758621
Average AUC: 0.571309045204
``

In [ ]:
def total_jaccard( train_x, train_y, test_x, test_y, model ):
    model.fit( train_x, train_y )
    p = model.predict( test_x )
    #print p
    p = (p >= 0.5).astype(np.float64)
    total = sum([jaccard_similarity_score(y_true, y_pred) for y_true, y_pred in zip(test_y, p)])
    print "Total Jaccard similarity:", total/len(test_x)

In [ ]:
with open(VOCAB_PATH, mode="wb") as vocab_file:
    for w in hatebase_data.index.values:
        vocab_file.write(w + b"\n")

In [ ]:
tf.reset_default_graph()
nn = OneLayerNN()
total_jaccard( train_x, train_y.iloc[:,:7], test_x, test_y.iloc[:,:7].values, nn )

In [ ]:
for i, field in enumerate(HATEBASE_FIELDS):
    print field
    tf.reset_default_graph()
    train_and_eval_auc( train_x, train_y.iloc[:,i], test_x, test_y.iloc[:,i], OneLayerNN() )

In [ ]:
lr = SoftmaxClassifier()

In [ ]:
total_jaccard( train_x, train_y.iloc[:,:7], test_x, test_y.iloc[:,:7].values, lr )

In [ ]:
for i, field in enumerate(HATEBASE_FIELDS):
    print field
    train_and_eval_auc( train_x, train_y.iloc[:,i], test_x, test_y.iloc[:,i], SoftmaxClassifier() )

Note: boundary for vectors passed to jaccard sim is 0.5

One Layer NN:

`
On 80-20 split
Iteration 1000: loss: 0.0963332206011 
Total Jaccard similarity: 0.862026862027
`


`about_class
Iteration 1000: loss: 0.184165582061 
AUC: 0.472934472934
about_ethnicity
Iteration 1000: loss: 0.0597166158259 
AUC: 0.564257028112
about_sexual_orientation
Iteration 1000: loss: 0.0233292710036 
AUC: 0.474358974359
about_religion
Iteration 1000: loss: 0.160992875695 
AUC: 0.525017618041
about_disability
Iteration 1000: loss: 0.168781414628 
AUC: 0.554943373125
about_gender
Iteration 1000: loss: 0.0206541772932 
AUC: 0.585034013605
about_nationality
Iteration 1000: loss: 0.0504829958081 
AUC: 0.578034682081
`

Softmax:

`
On 80-20 split
Iteration 1000: loss: 1.18320953846 
Total Jaccard similarity: 0.869352869353
`


`about_class
Iteration 1000: loss: 0.245341107249 
AUC: 0.459164292498
about_ethnicity
Iteration 1000: loss: 0.192264601588 
AUC: 0.70749665328
about_sexual_orientation
Iteration 1000: loss: 0.0931176915765 
AUC: 0.65483234714
about_religion
Iteration 1000: loss: 0.448324710131 
AUC: 0.570472163495
about_disability
Iteration 1000: loss: 0.430169701576 
AUC: 0.555096418733
about_gender
Iteration 1000: loss: 0.085390098393 
AUC: 0.296768707483
about_nationality
Iteration 1000: loss: 0.0770960450172 
AUC: 0.335260115607
`